In [1]:
from handnormalizer import normalize_text
from handnormalizer.parser.pokerstars_stats_parser import PokerStarsStatsParser, init_database

init_database()

parser = PokerStarsStatsParser(normalize_text)
parser.ingest_folder("../dataset")

HAND ID: 257581182379
HAND ID: 257581182655
HAND ID: 257581183704
HAND ID: 257581185714
HAND ID: 257581186190
HAND ID: 257581186398
HAND ID: 257581186685
HAND ID: 257581186747
HAND ID: 257581187338
HAND ID: 257581187339
HAND ID: 257581189047
HAND ID: 257581189408
HAND ID: 257581189805
HAND ID: 257581190259
HAND ID: 257581192612
HAND ID: 257581193631
HAND ID: 257581193967
HAND ID: 257581194103
HAND ID: 257581194285
HAND ID: 257581194413
HAND ID: 257581194497
HAND ID: 257581194550
HAND ID: 257581195015
HAND ID: 257581194990
HAND ID: 257581195163
HAND ID: 257581195735
HAND ID: 257581196793
HAND ID: 257581196906
HAND ID: 257581197155
HAND ID: 257581197243
HAND ID: 257581197705
HAND ID: 257581197959
HAND ID: 257581198521
HAND ID: 257581199079
HAND ID: 257581199142
HAND ID: 257581200271
HAND ID: 257581200499
HAND ID: 257581201569
HAND ID: 257581201858
HAND ID: 257581201918
HAND ID: 257581202335
HAND ID: 257581202320
HAND ID: 257581203116
HAND ID: 257581203321
HAND ID: 257581203652
HAND ID: 2

In [2]:
from handnormalizer.db.session import SessionLocal
from handnormalizer.db.models import Hand, PlayerHandStat

session = SessionLocal()

print("Hands:", session.query(Hand).count())
print("Player stats:", session.query(PlayerHandStat).count())

Hands: 18643
Player stats: 162639


In [4]:
ids = [h.site_hand_id for h in session.query(Hand).all()]
len(ids), len(set(ids))


(18643, 18643)

In [7]:
session = SessionLocal()
rows = session.query(PlayerHandStat).limit(5).all()
session.close()

rows

In [4]:
session = SessionLocal()
hand = session.query(Hand).first()
hand.__dict__


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7c5087449f10>,
 'hero_name': 'Hero',
 'id': 1,
 'site': 'PokerStars',
 'site_hand_id': '257581182379',
 'is_maximum_raise_2bet': True}

In [5]:
from handnormalizer.db.session import SessionLocal
from handnormalizer.db.models import Hand, PlayerHandStat

def show_hand_stats(site_hand_id: str):
    session = SessionLocal()
    hand = session.query(Hand).filter(Hand.site_hand_id == site_hand_id).first()
    if not hand:
        print("Hand not found")
        return
    
    print("HAND DATA:")
    print(hand.__dict__)

    stats = session.query(PlayerHandStat).filter(PlayerHandStat.hand_id == hand.id).all()
    print("\nPLAYER STATS:")
    for s in stats:
        print(s.__dict__)

# Ejemplo de uso:
show_hand_stats("257647774611")


HAND DATA:
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7c4ab22b4590>, 'site_hand_id': '257647774611', 'is_maximum_raise_2bet': True, 'hero_name': 'Hero', 'id': 6952, 'site': 'PokerStars'}

PLAYER STATS:
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7c4ab22b5310>, 'player_name': 'Player1', 'hand_id': 6952, 'checked_flop': False, 'bet_flop_any': False, 'id': 60245, 'folded_flop_any': False}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7c4ab22b5370>, 'player_name': 'Player2', 'hand_id': 6952, 'checked_flop': False, 'bet_flop_any': True, 'id': 60246, 'folded_flop_any': False}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7c4ab22b53d0>, 'player_name': 'Hero', 'hand_id': 6952, 'checked_flop': False, 'bet_flop_any': False, 'id': 60247, 'folded_flop_any': True}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7c4ab22b5430>, 'player_name': 'Player3', 'hand_id': 6952, 'che

In [11]:
from pathlib import Path

raw = Path("../dataset/Caph - €0.10 NL FAST (6 max) - 202508110733.txt").read_text(encoding="utf-8")
raw.splitlines()[0]


"\ufeffPokerStars Zoom Hand #257298513408:  Hold'em No Limit (€0.05/€0.10) - 2025/08/11 9:33:56 CET [2025/08/11 3:33:56 ET]"

In [16]:
from handnormalizer.parser.pokerstars_stats_parser import HAND_ID_PATTERN
HAND_ID_PATTERN.pattern


'^\\s*\\ufeff?PokerStars(?:\\s+Zoom)?\\s+Hand\\s+#(\\d+):'

In [17]:
from handnormalizer.parser.pokerstars_stats_parser import PokerStarsStatsParser
from handnormalizer import normalize_text

parser = PokerStarsStatsParser(normalize_text)

print(parser.extract_hand_id("\ufeffPokerStars Zoom Hand #257298513408: whatever"))
print(parser.extract_hand_id("PokerStars Zoom Hand #257298517166: whatever"))


257298513408
257298517166


In [19]:
from pathlib import Path
txt = Path("../dataset/Caph - €0.10 NL FAST (6 max) - 202508110733.txt").read_text(encoding="utf-8")
hands = txt.split("PokerStars")
hands[:3]   # show first pieces


['\ufeff',
 " Zoom Hand #257298513408:  Hold'em No Limit (€0.05/€0.10) - 2025/08/11 9:33:56 CET [2025/08/11 3:33:56 ET]\nTable 'Caph' 6-max Seat #1 is the button\nSeat 1: juanico1977 (€10.04 in chips)\nSeat 2: toshkogochev (€16.58 in chips)\nSeat 3: noarones (€8 in chips)\nSeat 4: kñamon (€3.69 in chips)\nSeat 5: titus&mus (€46.79 in chips)\nSeat 6: Mercimer75 (€10 in chips)\ntoshkogochev: posts small blind €0.05\nnoarones: posts big blind €0.10\n*** HOLE CARDS ***\nDealt to noarones [8d 8s]\nkñamon: raises €0.20 to €0.30\ntitus&mus: folds\nMercimer75: folds\njuanico1977: raises €0.60 to €0.90\ntoshkogochev: folds\nnoarones: folds\nkñamon: calls €0.60\n*** FLOP *** [Kd Ad Kc]\nkñamon: checks\njuanico1977: checks\n*** TURN *** [Kd Ad Kc] [5c]\nkñamon: checks\njuanico1977: bets €0.61\nkñamon: folds\nUncalled bet (€0.61) returned to juanico1977\njuanico1977 collected €1.85 from pot\njuanico1977: doesn't show hand\n*** SUMMARY ***\nTotal pot €1.95 | Rake €0.10\nBoard [Kd Ad Kc 5c]\nSeat 1:

In [20]:
raw_lines = txt.splitlines()
raw_lines[0], raw_lines[20], raw_lines[40]


("\ufeffPokerStars Zoom Hand #257298513408:  Hold'em No Limit (€0.05/€0.10) - 2025/08/11 9:33:56 CET [2025/08/11 3:33:56 ET]",
 'kñamon: checks',
 "PokerStars Zoom Hand #257298517166:  Hold'em No Limit (€0.05/€0.10) - 2025/08/11 9:34:56 CET [2025/08/11 3:34:56 ET]")

In [6]:
from pathlib import Path
from handnormalizer.parser.pokerstars_stats_parser import PokerStarsStatsParser, HAND_ID_PATTERN
from handnormalizer import normalize_text

def extract_ids_from_pt4_file(path: str):
    text = Path(path).read_text(encoding="utf-8")

    ids = []
    for line in text.splitlines():
        match = HAND_ID_PATTERN.search(line)
        if match:
            ids.append(match.group(1))

    return ids


In [7]:
from handnormalizer.db.session import SessionLocal
from handnormalizer.db.models import Hand, PlayerHandStat

def get_ids_from_parser_fold_flop():
    session = SessionLocal()
    rows = (
        session.query(Hand.site_hand_id)
        .join(PlayerHandStat)
        .filter(PlayerHandStat.player_name == Hand.hero_name)
        .filter(PlayerHandStat.folded_flop_any.is_(True))
        .all()
    )
    return [r[0] for r in rows]



In [8]:
def compare_id_lists(pt4_ids, parser_ids):
    pt4_set = set(pt4_ids)
    parser_set = set(parser_ids)

    missing_in_parser = pt4_set - parser_set
    extra_in_parser = parser_set - pt4_set

    if not missing_in_parser and not extra_in_parser:
        print("TODO OK! Exact match.")
        return

    if missing_in_parser:
        print("IDs missing in your parser (PT4 has them):", len(missing_in_parser))
        for x in sorted(missing_in_parser):
            print(x)

    if extra_in_parser:
        print("Extra IDs in parser (PT4 did not include them):", len(extra_in_parser))
        
        for x in sorted(extra_in_parser):
            print(x)


In [3]:
from PT4_testing.helpers import *

pt4_ids = extract_ids_from_pt4_file("folded_flop_any.txt")
parser_ids = get_ids_from_parser_fold_flop()

compare_id_lists(pt4_ids, parser_ids)


TODO OK! Exact match.


True